## Load and Visualize NeuronIO

In [1]:
import gc
import os
import sys
import h5py
import random
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

package_path = Path(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
sys.path.insert(0, str(package_path))

In [ ]:
from src.neuronio.neuronio_data_utils import (
    DEFAULT_Y_SOMA_THRESHOLD,
    DEFAULT_Y_TRAIN_SOMA_BIAS,
    DEFAULT_Y_TRAIN_SOMA_SCALE,
    NEURONIO_DATA_DIM,
    NEURONIO_LABEL_DIM,
    NEURONIO_SIM_LEN,
    NEURONIO_SIM_PER_FILE,
    create_neuronio_input_type,
    parse_sim_experiment_file,
    get_data_files_from_folder, 
)

In [ ]:
data_dir_path = Path("D:/NeuronIO").expanduser().resolve() # TODO: change to neuronio data path
train_data_dir_path = data_dir_path / "train"  # TODO: change to train subfolder
test_data_dir_path = data_dir_path / "test/Data_test"  # TODO: change to test subfolder

data_config = dict()
train_data_dirs = [
    str(train_data_dir_path / "full_ergodic_train_batch_1"),
    str(train_data_dir_path / "full_ergodic_train_batch_2"),
    str(train_data_dir_path / "full_ergodic_train_batch_3"),
    str(train_data_dir_path / "full_ergodic_train_batch_4"),
    str(train_data_dir_path / "full_ergodic_train_batch_5"),
    str(train_data_dir_path / "full_ergodic_train_batch_6"),
    str(train_data_dir_path / "full_ergodic_train_batch_7"),
    str(train_data_dir_path / "full_ergodic_train_batch_8"),
    str(train_data_dir_path / "full_ergodic_train_batch_9"),
    str(train_data_dir_path / "full_ergodic_train_batch_10"),
]
test_data_dirs = [str(test_data_dir_path)]

data_config["train_data_dirs"] = train_data_dirs
data_config["test_data_dirs"] = test_data_dirs

data_config["data_dim"] = NEURONIO_DATA_DIM 
data_config["label_dim"] = NEURONIO_LABEL_DIM

train_files = get_data_files_from_folder(data_config["train_data_dirs"])
test_files = get_data_files_from_folder(data_config["test_data_dirs"])

In [ ]:
ignore_time_from_start = 500
threshold = -65
threshold_h = -60
section_len = 200
start_point_persec = 20

In [ ]:
save_path = '../data_processed/NeuronIOstartpoint/'
for file_path in test_files:
    X, y_spike, y_soma = parse_sim_experiment_file(
        sim_experiment_file=file_path,
        include_params=False,
        )
    sim_len, sim_num = y_soma.shape
    num_sec = int(sim_len/section_len)
    sections = y_soma.T.reshape([sim_num,num_sec,section_len])
    start_time  = np.ones([sim_num, num_sec, start_point_persec], dtype=int)
    for sim in range(sim_num):
        for i in range(num_sec):
            choice = (sections[sim, i, :] < threshold).nonzero()[0]
            if len(choice) < start_point_persec:
                choice = (sections[sim, i, :] < threshold_h).nonzero()[0]
            try:
                start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=False)
            except:
                start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=True)
                print(file_path[-93:-2])
                print(len(choice))
            start_time[sim, i, :] += i * section_len
    
    start_time = start_time.reshape([sim_num, -1])
    np.save(save_path + file_path[-92:-2]+'.npy', start_time, allow_pickle=True)
    #print(file_path[-93:-2])

In [ ]:
start_time.shape

In [ ]:
train_files[1]

In [152]:
len(train_files)

300

In [141]:
sections = y_soma.T.reshape([sim_num,num_sec,section_len])
(sections < threshold_h).sum(axis=-1).min()

np.int64(65)

In [41]:
(y_soma[ignore_time_from_start:] < threshold).sum(axis=0).min()

np.int64(4223)

In [ ]:
# put them all to select
# select 4000 from each simulation

In [145]:
sections = y_soma.T.reshape([sim_num,num_sec,section_len])
start_time  = np.ones([sim_num, num_sec, start_point_persec], dtype=int)
for sim in range(sim_num):
    for i in range(num_sec):
        choice = (sections[sim, i, :] < threshold).nonzero()[0]
        if len(choice) < start_point_persec:
            choice = (sections[sim, i, :] < threshold_h).nonzero()[0]
        start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=False)
        start_time[sim, i, :] += i * section_len

start_time = start_time.reshape([sim_num, -1])

In [148]:
y_soma[start_time[0], 0]

array([-81.0625, -80.875 , -80.75  , -80.8125, -79.25  , -78.375 ,
       -81.    , -81.1875, -80.6875, -81.1875, -81.0625, -80.8125,
       -80.6875, -80.875 , -80.6875, -80.75  , -80.6875, -80.875 ,
       -78.6875, -80.9375, -80.75  , -81.    , -80.9375, -80.8125,
       -80.75  , -80.875 , -81.    , -80.9375, -80.875 , -80.75  ,
       -80.875 , -81.    , -80.8125, -80.6875, -80.75  , -80.9375,
       -80.8125, -80.8125, -80.8125, -80.75  , -80.875 , -80.75  ,
       -80.9375, -80.9375, -80.9375, -80.6875, -80.875 , -80.6875,
       -80.875 , -80.9375, -80.875 , -80.75  , -80.9375, -80.8125,
       -80.9375, -81.    , -80.75  , -80.9375, -80.875 , -80.9375,
       -81.0625, -80.875 , -80.9375, -81.0625, -80.9375, -81.0625,
       -81.    , -81.125 , -80.8125, -81.    , -80.875 , -81.125 ,
       -81.0625, -81.    , -81.0625, -80.875 , -80.9375, -81.125 ,
       -80.6875, -80.875 , -80.875 , -80.8125, -80.75  , -81.0625,
       -80.6875, -80.75  , -80.8125, -81.0625, -80.875 , -81.0

In [149]:
for sim in range(sim_num):
    print(y_soma[start_time[sim], sim].max())

-65.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.125
-60.15625
-65.1875
-65.0625
-65.0625
-65.0625
-60.40625
-65.125
-60.09375
-65.125
-65.0625
-65.375
-65.0625
-65.5
-65.0625
-60.0625
-65.0625
-65.1875
-65.0625
-65.0625
-65.0625
-60.03125
-65.375
-65.125
-60.5
-60.03125
-65.25
-60.03125
-60.0625
-65.3125
-65.0625
-60.03125
-60.0625
-60.125
-65.125
-65.0625
-65.0625
-65.0625
-60.0625
-60.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.1875
-65.9375
-65.0625
-65.0625
-60.28125
-65.0625
-65.0625
-65.0625
-60.15625
-65.0625
-65.0625
-65.125
-65.0625
-65.1875
-65.8125
-65.1875
-60.375
-60.03125
-65.125
-65.3125
-65.1875
-60.0625
-65.0625
-60.03125
-65.5
-65.0625
-65.0625
-60.9375
-60.15625
-65.125
-65.125
-60.0625
-60.625
-60.125
-60.5
-60.03125
-65.4375
-65.0625
-60.125
-60.125
-65.0625
-65.0625
-65.375
-65.1875
-65.0625
-60.03125
-69.5
-60.03125
-60.4375
-65.0625
-60.5
-65.125
-65.0625
-65.0625
-65.0625
-60.03125
-60.0625
-65.3125
-65.0625
-60.1875
-60.09375
-60.5
-65.0625
-60.96875
-60.531

## Generating Random Input

## Load Trained model

In [ ]:
# Initialize the ELM model
model = ELM(**model_config).to(torch_device)

# Load the best model for evaluation
state_dict = torch.load(general_config["artefacts_dir"] + "/neuronio_best_model_state.pt", map_location=torch_device)
print(model.load_state_dict(state_dict))
# Visualize ELM model
print(model)

## Generating Output from random input

In [ ]:
from src.neuronio.neuronio_eval_utils import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)

with torch.no_grad():
    test_predictions = compute_test_predictions_multiple_sim_files(
        neuron=model,
        test_files=test_files,
        burn_in_time=train_config["burn_in_time"],
        input_window_size=train_config["input_window_size"],
        device=torch_device,
    )
    test_results = filter_and_extract_core_results(*test_predictions, verbose=False)

eval_results = dict()
if general_config["train_valid_eval"]:
    eval_results["train_results"] = train_results
    eval_results["valid_results"] = valid_results
eval_results["test_results"] = test_results